# Theano  

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon even offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [1]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print theano.config.device # We're using CPUs (for now)
print theano.config.floatX # Should be 64 bit for CPUs

np.random.seed(0)

from IPython.display import display, clear_output 

cpu
float64


In [2]:
## basics: theano multiplication

a = T.scalar()
b = T.scalar()

y = a * b

mult = theano.function(inputs = [a, b], outputs = y)

print mult(1, 2)
print mult(4, 5)

2.0
20.0


In [3]:
# more arbitrary

# symbolic variables
a = T.scalar()
b = T.scalar()
c = T.scalar()
d = T.scalar()

# manipulation of these symbolic guys
z = a * b + c / d

# function actually does the work
complexGuy = theano.function(inputs= [a, b, c, d], outputs = z)

# here it is
print complexGuy(3, 3, 1, 5)

9.2


In [4]:
# shared variables: both symbolic and not...

w = theano.shared(np.asarray(0., dtype=theano.config.floatX))

xx = T.scalar()

update = [[w, w + xx]]

addition = theano.function(inputs=[xx], outputs=xx, updates=update)

# here's how you actually get it out
print w.get_value()

0.0


In [5]:
addition(1)
print w.get_value()

1.0


In [6]:
addition(4)
print w.get_value()

5.0


In [7]:
target_beta_1 = 4.
learning_rate = 0.01
noise = 0.2

trainX = np.linspace(-1, 1, 50)
trainY = target_beta_1 * trainX + np.random.randn(*trainX.shape) * noise

X = T.scalar()
Y = T.scalar()

def model(X, w):
    return X * w

w = theano.shared(np.asarray(0., dtype=theano.config.floatX))
yhat = model(X, w)

cost = T.mean(T.sqr(yhat - Y))
gradient = T.grad(cost=cost, wrt=w)

updates = [[w, w - gradient * learning_rate]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)

for i in range(50):
    for xx, yy in zip(trainX, trainY):
        cc = train(xx, yy)
        clear_output(wait=True)
        print "Round ", i, ":", w.get_value(), cc
        
print "final value: ", w.get_value()

Round  49 : 3.86382215238 0.0091280069454
final value:  3.86382215238


Can we add an intercept to the model above?

In [9]:
## intercept model here..
target_beta_1 = 13.
target_beta_0 = -2.
learning_rate = 0.01
noise = 0.2

trainX = np.linspace(-2, 1, 50)
trainY = target_beta_0 + target_beta_1 * trainX + np.random.randn(*trainX.shape) * noise

X = T.scalar()
Y = T.scalar()

def model(X, w):
    return X * w[1] + w[0]

w = theano.shared(np.asarray([0., 0.], dtype=theano.config.floatX))
yhat = model(X, w)

cost = T.mean(T.sqr(yhat - Y))
gradient = T.grad(cost=cost, wrt=w)

updates = [[w, w - gradient * learning_rate]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)

for i in range(50):
    for xx, yy in zip(trainX, trainY):
        cc = train(xx, yy)
        clear_output(wait=True)
        print "Round ", i, ":", w.get_value(), cc
        
print "final value: ", w.get_value()

Round  49 : [ -1.96462272  12.96159453] 0.00865069227087
final value:  [ -1.96462272  12.96159453]


Now for MNIST data...

In [10]:
# Repeating steps from Project 1 to prepare mnist dataset. 
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target
X = X / 255.0
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print 'Features = %d' %(numFeatures)
print 'Train set = %d' %(numTrainExamples)
print 'Test set = %d' %(numTestExamples)

Features = 784
Train set = 2000
Test set = 2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [11]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data

train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size

print 'Classes = %d' %(numClasses)

print '\n', train_labels_b[:10, :], '\n'
print train_labels[:10], '\n'

Classes = 10

[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]] 

[ 8.  0.  6.  6.  8.  4.  0.  3.  8.  9.] 



Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [12]:
neighbors = 1

knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)

print 'Train time = %.2f' %(time.time() - start_time)

start_time = time.time()
accuracy = knn.score(test_data, test_labels)

print 'Accuracy = %.4f' %(accuracy)
print 'Prediction time = %.2f' %(time.time() - start_time)

Train time = 0.31
Accuracy = 0.9170
Prediction time = 17.88


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

In [13]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, values.
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

# print *(numFeatures, numClasses)

Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

In [14]:
## (2) Model
# Theano objects accessed with standard Python variables
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))

y_hat = model(X, w)

We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

In [15]:
## (3) Cost function
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [16]:
## (4) Objective (and solver)

alpha = 0.01

gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) # computes cost, then runs update

y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        
        cost = train(train_data[0:len(train_data)], train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescent(500)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

500) accuracy = 0.8320
train time = 17.35
predict time = 0.01


Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [65]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 5
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))  
        
    print 'train time = %.2f' %(trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

50) accuracy = 0.8710
train time = 65.99
predict time = 0.00


Exercise: What do you expect to happen if you switch the batch size to be greater than 1 (i.e. mini-batch)?  Why?

Try it for a few values...

In [18]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))) 
        
    print 'train time = %.2f' %(trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

50) accuracy = 0.8915
train time = 57.89
predict time = 0.00


#### PART 2: Multi-layer Neural Networks

As we start to get further into Neural Networks, if you'd like to take time on your own for an in-depth introduction on the state of the art in the topic, check out this excellent 1-day tutorial from KDD2014:

Part 1: http://videolectures.net/kdd2014_bengio_deep_learning/

Part 2: http://videolectures.net/tcmm2014_taylor_deep_learning/

---------

Let's take our implementation of logistic regression (which recall is in fact a single layer neural network), and add a hidden layer, making it a two layer neural network.  Because we have a hidden layer, we will now train the model using backpropagation.

Exercise: How do you expect this model to compare to KNN and logistic regression in terms of train time and accuracy?  Why?

Let's try it out...

In [71]:
## (1) Parameters
numHiddenNodes = 50 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]

## (2) Model
X = T.matrix()
Y = T.matrix()

# Two notes:
# First, feed forward is the composition of layers (dot product + activation function)
# Second, activation on the hidden layer still uses sigmoid
def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2))
y_hat = model(X, w_1, w_2)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Minimization.  Update rule changes to backpropagation. (the chain rule)
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates

update = backprop(cost, params)

# why not this??
# grad = T.grad(cost=cost, wrt=params)
# update = [[params, params - grad * alpha]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(300)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

300) accuracy = 0.8870
train time = 3863.10
predict time = 0.01


Exercise: Change the number of nodes in the hidden layer?  What do you expect the impact to be?  What is the impact?

--------

As interest in networks with more layers and more complicated architechures has increased, a couple of tricks have emerged and become standard practice.  Let's look at two of those--rectifier activation and dropout noise.

Exercise:  We saw an improvement from adding a hidden layer.  What do you expect to happen if a second hidden layer was added?  

Let's try it...

In [76]:
## (1) Parms
numHiddenNodes = 50 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w_1, w_2, w_3):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2)), w_3))
y_hat = model(X, w_1, w_2, w_3)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.1
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates

update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(1000)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

282) accuracy = 0.8825


KeyboardInterrupt: 

#### Activation Revisted

Let's look at a recent idea around activation closely associated with deep learning.  In 2010, in a paper published at NIPS (https://www.utc.fr/~bordesan/dokuwiki/_media/en/glorot10nipsworkshop.pdf), Yoshua Bengio showed that rectifier activation works better empirically than sigmoid activation when used in the hidden layers.  

The rectifier activation is simple: f(x)=max(0,x).  Intuitively, the difference is that as a sigmoid activated node approaches 1 it stops learning even if error continues to be propagated to it, whereas the rectifier activated node continue to learn (at least in the positive direction).  It is not completely understood (per Yoshua Bengio) why this helps, but there are some theories being explored including as related to the benefits of sparse representations in networks. (http://www.iro.umontreal.ca/~bengioy/talks/KDD2014-tutorial.pdf).  Rectifiers also speed up training.

Although the paper was published in 2010, the technique didn't gain widespread adoption until 2012 when members of Hinton's group spread the word, including with this Kaggle entry: http://blog.kaggle.com/2012/11/01/deep-learning-how-i-did-it-merck-1st-place-interview/

Let's change the activation in our 2 layer network to rectifier and see what happens...

In [79]:
## (1) Parms
numHiddenNodes = 200 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()

def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.maximum(T.dot(X, w_1), 0.), w_2))
y_hat = model(X, w_1, w_2)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.05
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates

update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(100)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

100) accuracy = 0.8965
train time = 1098.43
predict time = 0.03


#### Maxout Activation

So rectifier activation worked great!  

Exercise: try another type of activation called Maxout (or Max Pooling) activiation.  Maxout activation just selects the max input as the output.  Maxout is a type of pooling, a technique which performs particularly well for vision problems. For more background see: http://jmlr.org/proceedings/papers/v28/goodfellow13.pdf and http://www.quora.com/What-is-impact-of-different-pooling-methods-in-convolutional-neural-networks  

#### Noise

Previously when working with the MNIST data we saw a benefit in generalization from adding noise to the training data.  Let's try that again here, however this time with a trick for adding noise called 'Dropouts'.  The idea with dropouts is that instead of (or in addition to) adding noise to our inputs, we add noise by having each node return 0 with a certain probability during training.  This trick both improves generalization in large networks and speeds up training.

Hinton introduced the idea in 2012 and gave an explanation of why it's similar to bagging (http://arxiv.org/pdf/1207.0580v1.pdf)

Let's give it a try...

In [ ]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parms
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()
srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, p_1, p_2):
    return T.nnet.softmax(T.dot(dropout(T.maximum(T.dot(dropout(X, p_1), w_1),0.), p_2), w_2))

y_hat_train = model(X, w_1, w_2, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, 0., 0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)


miniBatchSize = 10
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

Let's try once again adding a second hidden layer...

In [18]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parmeters
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]


## (2) Model
X = T.matrix()
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, w_3, p_1, p_2, p_3):
    return T.nnet.softmax(T.dot(dropout(T.maximum(T.dot(dropout(T.maximum(T.dot(dropout(X, p_1), w_1),0.), p_2), w_2),0.), p_3), w_3))

y_hat_train = model(X, w_1, w_2, w_3, 0.2, 0.5,0.5)
y_hat_predict = model(X, w_1, w_2, w_3, 0., 0.,0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(10)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

1) accuracy = 0.5940
2) accuracy = 0.8105
3) accuracy = 0.8515
4) accuracy = 0.8705
5) accuracy = 0.8840
6) accuracy = 0.8840
7) accuracy = 0.8890
8) accuracy = 0.9070
9) accuracy = 0.9015
10) accuracy = 0.8975
train time = 518.44
predict time = 0.18


#### Part 3: Convolutional Neural Nets

Today, when the phrase 'deep learning' is used to describe a system, it is often a convolution neural network (or convonet).  The convonet architechture was largely developed in the late 90's at Bell Labs, but only very recently popularized.  It was developed for image recognition, and is described and implemented with 2d representations in mind.

Geoffrey Hinton has an excellent two-part lecture on the topic:

https://www.youtube.com/watch?v=6oD3t6u5EPs

https://www.youtube.com/watch?v=fueIAeAsGzA

Also, this code was partly taken from these tutorials, which are worth referring back to:

http://deeplearning.net/tutorial/lenet.html

http://ufldl.stanford.edu/tutorial/supervised/FeatureExtractionUsingConvolution/

https://www.youtube.com/watch?v=S75EdAcXHKk

http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

In [ ]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

## (1) Parameters
numHiddenNodes = 600 
patchWidth = 3
patchHeight = 3
featureMapsLayer1 = 32
featureMapsLayer2 = 64
featureMapsLayer3 = 128

# For convonets, we will work in 2d rather than 1d.  The MNIST images are 28x28 in 2d.
imageWidth = 28
train_data = train_data.reshape(-1, 1, imageWidth, imageWidth)
test_data = test_data.reshape(-1, 1, imageWidth, imageWidth)

# Convolution layers.  
w_1 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer1, 1, patchWidth, patchHeight))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer2, featureMapsLayer1, patchWidth, patchHeight))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer3, featureMapsLayer2, patchWidth, patchHeight))*.01)))

# Fully connected NN. 
w_4 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer3 * 3 * 3, numHiddenNodes))*.01)))
w_5 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3, w_4, w_5]

## (2) Model
X = T.tensor4() # conv2d works with tensor4 type
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

# Theano provides built-in support for add convolutional layers
def model(X, w_1, w_2, w_3, w_4, w_5, p_1, p_2):
    l1 = dropout(max_pool_2d(T.maximum(conv2d(X, w_1, border_mode='full'),0.), (2, 2)), p_1)
    l2 = dropout(max_pool_2d(T.maximum(conv2d(l1, w_2), 0.), (2, 2)), p_1)
    l3 = dropout(T.flatten(max_pool_2d(T.maximum(conv2d(l2, w_3), 0.), (2, 2)), outdim=2), p_1) # flatten to switch back to 1d layers
    l4 = dropout(T.maximum(T.dot(l3, w_4), 0.), p_2)
    return T.nnet.softmax(T.dot(l4, w_5))

y_hat_train = model(X, w_1, w_2, w_3, w_4, w_5, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, w_3, w_4, w_5, 0., 0.)
y_x = T.argmax(y_hat, axis=1)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  
def backprop(cost, w, alpha=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        
        # adding gradient scaling
        acc = theano.shared(w1.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * grad ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        grad = grad / gradient_scaling
        updates.append((acc, acc_new))
        
        updates.append((w1, w1 - grad * alpha))
    return updates

update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescentStochastic(10)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

#### Additional topic: Brain inspiration

The architechture of the convonet was inspired by the visual cortext in the human brain.  If you are interested in learning more, check out: http://www-psych.stanford.edu/~ashas/Cognition%20Textbook/chapter2.pdf

#### Additional topic: Self-Driving Vehicles

Convolutional networks are starting to be used more and more for self-driving vehicles.  This past year, NVIDEA started produced a chipset for self-driving vehicles that analyzes the video of up to 14 cameras using convonets for 

CES 2015 parts 6,7,9

https://www.youtube.com/watch?v=-vKGkxeflGw

https://www.youtube.com/watch?v=zsVsUvx8ieo

https://www.youtube.com/watch?v=RvQVyGOynFY

GTC 2015 parts 4,5,7,9

https://www.youtube.com/watch?v=pqvdZ2jp1NA

https://www.youtube.com/watch?v=GGxdP_JWhwI

https://www.youtube.com/watch?v=Tb7ZYSTYHbw

https://www.youtube.com/watch?v=TDm6Snkle70